## Project - Course Booking AI Asssistant
AI Customer Support Bot that 
- Returns Prices
- Books Tickets
- Adds Information to Text File

In [0]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [0]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [0]:
system_message = "You are a helpful assistant for an Online Course Platform called StudyAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "If you are given a partial name, for example 'discrete' instead of 'discrete structures' \
ask the user if they meant to say 'discrete structures', and then display the price. The user may also use \
acronyms like 'PF' instead of programming fundamentals or 'OOP' to mean 'Object oriented programming'. \
Clarify what the user means and then proceed as directed."

In [0]:
course_prices = {"programming fundamentals": "$19", "discrete structures": "$39", "operating systems": "$24", "object oriented programming": "$39"}

def get_course_price(course):
    print(f"Tool get_course_price called for {course}")
    course = course.lower()
    return course_prices.get(course, "Unknown")

def enroll_in_course(course):
    print(f'Tool enroll_in_course_ called for {course}')
    course_price = get_course_price(course)
    if course_price != 'Unknown':
        with open('enrolled_courses.txt', 'a') as file:  
            file.write(course + "\n")
        return 'Successfully enrolled in course'
    else:
        return 'Enrollment failed, no such course available'

In [0]:
get_course_price('graph theory')
get_course_price('discrete structures')

In [0]:
price_function = {
    "name": "get_course_price",
    "description": "Get the price of a course. Call this whenever you need to know the course price, for example when a customer asks 'How much is a ticket for this course?'",
    "parameters": {
        "type": "object",
        "properties": {
            "course": {
                "type": "string",
                "description": "The course that the customer wants to purchase",
            },
        },
        "required": ["course"],
        "additionalProperties": False
    }
}

enroll_function = {
    "name": "enroll_in_course",
    "description":"Get the success status of course enrollment. Call whenever a customer wants to enroll in a course\
                for example, if they say 'I want to purchase this course' or 'I want to enroll in this course'",
    "parameters":{
        "type":"object",
        "properties":{
            "course":{
                "type":"string",
                "description": "The course that the customer wants to purchase",
            },
        },
        "required": ["course"],
        "additionalProperties": False
    }   
}

In [0]:
tools = [
    {"type":"function","function":price_function},
    {"type":"function","function":enroll_function}
]

In [0]:
def chat(message,history):
    messages = [{"role":"system","content":system_message}] + history + [{"role":"user","content":message}]
    response = openai.chat.completions.create(model=MODEL,messages=messages,tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        messages.append(message)
        for tool_call in message.tool_calls:
            messages.append(handle_tool_call(tool_call))
        response = openai.chat.completions.create(model=MODEL,messages=messages)

    return response.choices[0].message.content

In [0]:
# We have to write that function handle_tool_call:

def handle_tool_call(tool_call):
    function = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    match function:
        case 'get_course_price':
            course = arguments.get('course')
            price = get_course_price(course)
            return {
                "role": "tool",
                "content": json.dumps({"course": course,"price": price}),
                "tool_call_id": tool_call.id
            }
        case 'enroll_in_course':
            course = arguments.get('course')
            status = enroll_in_course(course)
            return {
                "role": "tool",
                "content": json.dumps({"course": course, "status": status}),
                "tool_call_id": tool_call.id
            }
    

In [0]:
gr.ChatInterface(fn=chat,type="messages").launch(inbrowser=True)